# Week 2 Exercise — Code Explainer Chatbot

A full Gradio prototype of the technical question-answerer from Week 1.

**Features:**
- Gradio UI with `gr.Blocks`
- Streaming responses
- Expert system prompt
- Model switching (Claude 3.5 Sonnet ↔ Llama 3.2)
- Tool calling: `lookup_python_docs`

In [1]:
import os
import json
from dotenv import load_dotenv
from anthropic import Anthropic
import ollama
import gradio as gr

In [2]:
# Load environment variables
load_dotenv(override=True)

api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    import getpass
    print('ANTHROPIC_API_KEY not found. Please enter it:')
    api_key = getpass.getpass()
    os.environ['ANTHROPIC_API_KEY'] = api_key

anthropic_client = Anthropic()
print(f'Anthropic API Key loaded (starts with {api_key[:12]}...)')

Anthropic API Key loaded (starts with sk-ant-api03...)


In [4]:
# Model options
MODEL_CLAUDE = 'claude-sonnet-4-6'
MODEL_LLAMA = 'llama3.2'
MODEL_CHOICES = [MODEL_CLAUDE, MODEL_LLAMA]

In [5]:
SYSTEM_PROMPT = """
You are a senior Python engineer and educator.
When a user shares a code snippet or asks a technical question:

1. Identify the key constructs and patterns used.
2. Explain what the code does in plain English.
3. Walk through the execution step-by-step.
4. Highlight any potential pitfalls or best-practice improvements.

Keep your explanations clear and concise. Use markdown formatting for readability.
If you need to look up documentation for a Python built-in, use the lookup_python_docs tool.
"""

In [6]:
# Tool: lookup_python_docs
# Returns the official Python documentation URL for common built-ins

PYTHON_DOCS = {
    'yield': 'https://docs.python.org/3/reference/expressions.html#yield-expressions',
    'yield from': 'https://docs.python.org/3/reference/expressions.html#yield-expressions',
    'list comprehension': 'https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions',
    'dict comprehension': 'https://docs.python.org/3/tutorial/datastructures.html#dictionaries',
    'set comprehension': 'https://docs.python.org/3/tutorial/datastructures.html#sets',
    'generator': 'https://docs.python.org/3/howto/functional.html#generators',
    'decorator': 'https://docs.python.org/3/glossary.html#term-decorator',
    'lambda': 'https://docs.python.org/3/reference/expressions.html#lambda',
    'map': 'https://docs.python.org/3/library/functions.html#map',
    'filter': 'https://docs.python.org/3/library/functions.html#filter',
    'reduce': 'https://docs.python.org/3/library/functools.html#functools.reduce',
    'zip': 'https://docs.python.org/3/library/functions.html#zip',
    'enumerate': 'https://docs.python.org/3/library/functions.html#enumerate',
    'async': 'https://docs.python.org/3/library/asyncio.html',
    'await': 'https://docs.python.org/3/library/asyncio-task.html#awaitables',
    'dataclass': 'https://docs.python.org/3/library/dataclasses.html',
    'namedtuple': 'https://docs.python.org/3/library/collections.html#collections.namedtuple',
}

def lookup_python_docs(topic):
    """Return the Python docs URL for a given topic."""
    topic_lower = topic.lower().strip()
    url = PYTHON_DOCS.get(topic_lower)
    if url:
        return f'Documentation for **{topic}**: {url}'
    return f'No specific documentation link found for "{topic}". Try searching at https://docs.python.org/3/search.html?q={topic}'

# Anthropic tool schema
docs_tool = {
    'name': 'lookup_python_docs',
    'description': 'Look up the official Python documentation URL for a given Python concept, built-in, or keyword.',
    'input_schema': {
        'type': 'object',
        'properties': {
            'topic': {
                'type': 'string',
                'description': 'The Python concept or keyword to look up, e.g. yield, lambda, decorator',
            },
        },
        'required': ['topic'],
    },
}
tools = [docs_tool]

In [7]:
def handle_tool_calls(message):
    """Process tool calls from the assistant message and return tool responses."""
    responses = []
    for content_block in message.content:
        if content_block.type == 'tool_use' and content_block.name == 'lookup_python_docs':
            topic = content_block.input.get('topic', '')
            result = lookup_python_docs(topic)
            responses.append({
                'type': 'tool_result',
                'tool_use_id': content_block.id,
                'content': result,
            })
    return responses

In [8]:
def chat(message, history, model_name):
    """Chat callback for Gradio. Streams responses from the selected model."""
    history = [{'role': h['role'], 'content': h['content']} for h in history]

    if model_name == MODEL_CLAUDE:
        # --- Anthropic path (with tool calling) ---
        messages_for_claude = history + [{'role': 'user', 'content': message}]
        
        # First, check if the model wants to call a tool (non-streaming)
        response = anthropic_client.messages.create(
            model=MODEL_CLAUDE, 
            system=SYSTEM_PROMPT,
            max_tokens=1024,
            messages=messages_for_claude, 
            tools=tools
        )
        
        if response.stop_reason == 'tool_use':
            tool_responses = handle_tool_calls(response)
            messages_for_claude.append({'role': 'assistant', 'content': response.content})
            messages_for_claude.append({'role': 'user', 'content': tool_responses})
            
            # Now stream the final answer after tool use
            with anthropic_client.messages.stream(
                model=MODEL_CLAUDE, 
                system=SYSTEM_PROMPT,
                max_tokens=1024,
                messages=messages_for_claude, 
                tools=tools
            ) as stream:
                result = ''
                for text in stream.text_stream:
                    result += text
                    yield result
        else:
            # Model didn't use a tool, so we can just return the content (we could also stream this directly)
             yield response.content[0].text

    else:
        # --- Ollama path (streaming, no tool calling) ---
        messages = [{'role': 'system', 'content': SYSTEM_PROMPT}] + history + [{'role': 'user', 'content': message}]
        stream = ollama.chat(
            model=MODEL_LLAMA,
            messages=messages,
            stream=True,
        )
        result = ''
        for chunk in stream:
            result += chunk['message']['content']
            yield result

In [ ]:
# Launch the Gradio UI

with gr.Blocks(title='Code Explainer Chatbot') as ui:
    gr.Markdown('##Code Explainer Chatbot')
    gr.Markdown('Paste a code snippet and get a step-by-step explanation. Switch models anytime.')

    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=MODEL_CHOICES,
            value=MODEL_CLAUDE,
            label='Select Model',
        )

    chatbot = gr.ChatInterface(
        fn=chat,
        type='messages',
        additional_inputs=[model_dropdown],
    )

ui.launch(share=True)